In [2]:
import numpy as np
import pandas as pd
from scipy.stats import ttest_ind
from statsmodels.stats.proportion import proportions_ztest
# import geopandas as gpd
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter

In [3]:
state_df = pd.read_csv('./StatewiseTestingDetails.csv')
country_df = pd.read_csv('./covid_19_india.csv')

In [4]:
state_df.head()

,Date,State,TotalSamples,Negative,Positive
0,2020-04-17,Andaman and Nicobar Islands,1403.0,1210,12.0
1,2020-04-24,Andaman and Nicobar Islands,2679.0,NaN,27.0
2,2020-04-27,Andaman and Nicobar Islands,2848.0,NaN,33.0
3,2020-05-01,Andaman and Nicobar Islands,3754.0,NaN,33.0
4,2020-05-16,Andaman and Nicobar Islands,6677.0,NaN,33.0


In [5]:
#Copying the contry_df data to df for easy reference
df=country_df.copy()

In [6]:
#Keeping it simple, we will drop the rows with state name ending with "***" as it is seems to be rows with incomplete values.
def drop_star(df):
    for i in df['State/UnionTerritory'].iteritems():
        if i[1][-3:] == "***":
            df.drop(i[0],inplace=True)
        
drop_star(df)
States=df['State/UnionTerritory'].unique()

In [7]:
#lets convert the Date feature to Date&time datatype
df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d')

#Time is not required as it doesnt make much difference
df.drop(['Time'],axis=1, inplace=True)

#Renaming State/UnionTerritory to States for easy reference
df.rename(columns={'State/UnionTerritory':'States'}, inplace=True)

In [8]:
df['Active_cases']=df['Confirmed']-(df['Cured']+df['Deaths'])
df['Discharge_Rate'] = np.round((df['Cured']/df['Confirmed'])*100, decimals = 4)      # create instances for 'death_rate and discharge_rate'
df['Death_Rate'] = np.round((df['Deaths']/df['Confirmed'])*100, decimals = 4)
df.head()

,Sno,Date,States,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,Active_cases,Discharge_Rate,Death_Rate
0,1,2020-01-30,Kerala,1,0,0,0,1,1,0.0,0.0
1,2,2020-01-31,Kerala,1,0,0,0,1,1,0.0,0.0
2,3,2020-02-01,Kerala,2,0,0,0,2,2,0.0,0.0
3,4,2020-02-02,Kerala,3,0,0,0,3,3,0.0,0.0
4,5,2020-02-03,Kerala,3,0,0,0,3,3,0.0,0.0


In [9]:
#Current date from the record
current = df[df.Date == '2021-07-06']

In [10]:
df.tail()

,Sno,Date,States,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,Active_cases,Discharge_Rate,Death_Rate
18105,18106,2021-08-11,Telangana,-,-,638410,3831,650353,8112,98.1636,0.5891
18106,18107,2021-08-11,Tripura,-,-,77811,773,80660,2076,96.4679,0.9583
18107,18108,2021-08-11,Uttarakhand,-,-,334650,7368,342462,444,97.7189,2.1515
18108,18109,2021-08-11,Uttar Pradesh,-,-,1685492,22775,1708812,545,98.6353,1.3328
18109,18110,2021-08-11,West Bengal,-,-,1506532,18252,1534999,10215,98.1455,1.1891


In [11]:
df['Date']= pd.to_datetime(df['Date'])          # Date is converted to DateTime format.
df.tail()

,Sno,Date,States,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,Active_cases,Discharge_Rate,Death_Rate
18105,18106,2021-08-11,Telangana,-,-,638410,3831,650353,8112,98.1636,0.5891
18106,18107,2021-08-11,Tripura,-,-,77811,773,80660,2076,96.4679,0.9583
18107,18108,2021-08-11,Uttarakhand,-,-,334650,7368,342462,444,97.7189,2.1515
18108,18109,2021-08-11,Uttar Pradesh,-,-,1685492,22775,1708812,545,98.6353,1.3328
18109,18110,2021-08-11,West Bengal,-,-,1506532,18252,1534999,10215,98.1455,1.1891


In [12]:
from fbprophet import Prophet
active = df.groupby(['States', 'Date']).sum()['Active_cases'].reset_index()
active.tail()

ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
active.columns = ['States','ds', 'y']
active['ds'] = pd.to_datetime(active['ds'])

In [ ]:
all_states = active['States'].unique()
for state in all_states:
    globals()[state] = active[active['States']==state]

In [ ]:
enter_state = input("Enter the State: ")
enter_state

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(globals()[enter_state])
future = m.make_future_dataframe(periods=53)       #Making future prediction for next 60 days
future.tail(10)

In [ ]:
forecast = m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
last_active_pred = forecast[['yhat']].iloc[-1]
last_active_pred=int(last_active_pred)
print("Last active ", last_active_pred)

In [ ]:
active_forecast_plot = m.plot(forecast)

# RISK CALCULATED

In [ ]:
risk=0
globals()[f"mean_{enter_state}"] = np.mean(globals()[enter_state].y)
if last_active_pred*0.3 < globals()[f"mean_{enter_state}"]:
    risk=0
elif last_active_pred*0.6 < globals()[f"mean_{enter_state}"]:
    risk=1
else :
    risk=2

In [ ]:
risk